In [ ]:
# ==============================================
# STEP 1: Mount Google Drive
# ==============================================
from google.colab import drive
drive.mount('/content/gdrive')

# Define root path
ROOT_DIR = '/content/gdrive/My Drive/CAMS_'


# ==============================================
# STEP 2: Install and Import YOLOv8 (Ultralytics)
# ==============================================
!pip install ultralytics --quiet

import os
from ultralytics import YOLO
import pandas as pd


# ==============================================
# STEP 3: Dataset structure check
# Your directory must look like:
# CAMS_/
#   ├── train/
#   │    ├── images/
#   │    └── labels/
#   ├── val/  <-- This should be 'valid' based on output
#   │    ├── images/
#   │    └── labels/
#   └── data.yaml
# ==============================================

# Example content of data.yaml (edit inside CAMS_):
# train: /content/gdrive/My Drive/CAMS_/train/images
# val: /content/gdrive/My Drive/CAMS_/val/images # This should be 'valid' based on output
# names:
#   0: attentive
#   1: non_attentive


# ==============================================
# STEP 4: Load pretrained model (transfer learning)
# ==============================================
# Instead of training from scratch (yolov8n.yaml), use pretrained weights.
model = YOLO("yolov8n.pt")  # pre-trained on COCO dataset

# ==============================================
# STEP 5: Train the model with strong augmentations
# ==============================================
results = model.train(
    data=os.path.join(ROOT_DIR, "data.yaml"),
    epochs=100,           # you can increase to 150–200 for better results
    batch=16,
    imgsz=640,
    lr0=0.001,            # learning rate
    optimizer='Adam',     # better convergence
    pretrained=True,      # ensures pretrained backbone
    patience=20,          # stop early if no improvement
    augment=True,         # enable strong augmentations
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # color jitter
    degrees=10.0, translate=0.1, scale=0.5, shear=2.0, flipud=0.2, fliplr=0.5,
)

# ==============================================
# STEP 6: Validate model performance
# ==============================================
metrics = model.val()
print("📊 Validation metrics:")
print(metrics)


# ==============================================
# STEP 7: Make predictions on validation/test set
# ==============================================
preds = model.predict(
    source=os.path.join(ROOT_DIR, "valid/images"), # Changed 'val' to 'valid'
    save=True,  # saves annotated images
    conf=0.4,   # confidence threshold
    iou=0.45    # intersection threshold
)


# ==============================================
# STEP 8: Generate CSV report of predictions
# ==============================================
results_list = []

for r in preds:
    image_name = os.path.basename(r.path)
    if not hasattr(r, 'boxes'):
        continue
    for box in r.boxes:
        conf = float(box.conf)
        cls = int(box.cls)
        label = model.names[cls]
        results_list.append({
            "image": image_name,
            "attentive_prob": conf if label == "attentive" else 1 - conf,
            "non_attentive_prob": 1 - conf if label == "attentive" else conf,
            "status": label
        })

df = pd.DataFrame(results_list)
csv_path = os.path.join(ROOT_DIR, "attendance_predictions.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV file saved at: {csv_path}")

# ==============================================
# STEP 9: Backup training results to Drive
# ==============================================
!cp -r /content/runs /content/gdrive/My\ Drive/CAMS_/runs_backup
print("📂 Training runs backed up successfully to Google Drive!")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/My Drive/CAMS_/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False